In [1]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
import sys
path="data/"
inputname="human.fasta"
outputname="human_PSDSP"

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
'''
cross validation svm
'''
classifier="SVM"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV

y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=0
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    
    print len(positive_seq[0])
    print np.array(final_seq_value).shape
    final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in final_seq_value]
    print np.array(final_seq_value).shape
    y_final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in y_final_seq_value]
    print np.array(y_final_seq_value).shape

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    C=clf.best_params_['C']
    y_pred_prob=clf.predict_proba(X_test)
    
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    y_pred=clf.predict(X_test)
    
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save("svm_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.880530973451
ACC_all: 0.880530973451
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.853982300885
ACC_all: 1.73451327434
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.893805309735
ACC_all: 2.62831858407
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.898230088496
ACC_all: 3.52654867257
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.898230088496
ACC_all: 4.42477876106
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.920353982301
ACC_all: 5.34513274336
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.91592920354
ACC_all: 6.2610619469
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.933628318584
ACC_all: 7.19469026549
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.91592920354
ACC_all: 8.11061946903
42
(2034, 40)
(2034, 38)
(226, 38)
c: 32.0 gamma: 4.0
0.946902654867
ACC_all: 9.05752212389
[[['svmC:32.0gamma:4.0', 0.90575221238938042, 1.0, 0.8115044247787611, 0.811504424

True

In [10]:
'''
cross validation GBDT
'''
classifier="GBDT"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    
    final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in final_seq_value]
    print np.array(final_seq_value).shape

    y_final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in y_final_seq_value]
    print np.array(y_final_seq_value).shape
#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    
    GBDT = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
    parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
    clf = GridSearchCV(GBDT, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    n_estimators=clf.best_params_['n_estimators']
    max_depth=clf.best_params_['max_depth']
    min_samples_split=clf.best_params_['min_samples_split']
    min_samples_leaf=clf.best_params_['min_samples_leaf']
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf)
            ,ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

(2034, 37)
(226, 37)
0.880530973451
ACC_all: 0.880530973451
(2034, 37)
(226, 37)
0.849557522124
ACC_all: 1.73008849558
(2034, 37)
(226, 37)
0.862831858407
ACC_all: 2.59292035398
(2034, 37)
(226, 37)
0.893805309735
ACC_all: 3.48672566372
(2034, 37)
(226, 37)
0.898230088496
ACC_all: 4.38495575221
(2034, 37)
(226, 37)
0.920353982301
ACC_all: 5.30530973451
(2034, 37)
(226, 37)
0.898230088496
ACC_all: 6.20353982301
(2034, 37)
(226, 37)
0.929203539823
ACC_all: 7.13274336283
(2034, 37)
(226, 37)
0.902654867257
ACC_all: 8.03539823009
(2034, 37)
(226, 37)
0.946902654867
ACC_all: 8.98230088496
[[['GBDTn_estimators:111max_depth:3min_samples_split:700min_samples_leaf:90', 0.89823008849557517, 0.982511780901253, 0.8115044247787611, 0.8115044247787611, 0.9849557522123893, 0.8935120174117195, 0.88779431482650017, 0.88779431482650017, 0.80974167474378755, 0.9158101652439502, 917.0, 213.0, 17.0, 1113.0, 1130.0, 1130.0]]]


True

In [11]:
'''
cross validation RF
'''
classifier="RF"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    print np.array(final_seq_value).shape
    print len(positive_seq[0])
    final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in final_seq_value]
    print np.array(final_seq_value).shape

    y_final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in y_final_seq_value]

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    
    RF = RandomForestClassifier(min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
    parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
    clf = GridSearchCV(RF, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    n_estimators=clf.best_params_['n_estimators']
    max_depth=clf.best_params_['max_depth']
    min_samples_split=clf.best_params_['min_samples_split']
    min_samples_leaf=clf.best_params_['min_samples_leaf']
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf)
            ,ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

(2034, 39)
42
(2034, 37)
0.849557522124
ACC_all: 0.849557522124
(2034, 39)
42
(2034, 37)
0.840707964602
ACC_all: 1.69026548673
(2034, 39)
42
(2034, 37)
0.681415929204
ACC_all: 2.37168141593
(2034, 39)
42
(2034, 37)
0.880530973451
ACC_all: 3.25221238938
(2034, 39)
42
(2034, 37)
0.893805309735
ACC_all: 4.14601769912
(2034, 39)
42
(2034, 37)
0.907079646018
ACC_all: 5.05309734513
(2034, 39)
42
(2034, 37)
0.907079646018
ACC_all: 5.96017699115
(2034, 39)
42
(2034, 37)
0.911504424779
ACC_all: 6.87168141593
(2034, 39)
42
(2034, 37)
0.907079646018
ACC_all: 7.77876106195
(2034, 39)
42
(2034, 37)
0.933628318584
ACC_all: 8.71238938053
[[['RFn_estimators:32max_depth:3min_samples_split:700min_samples_leaf:60', 0.8712389380530976, 0.9955555555555555, 0.7442477876106195, 0.7442477876106195, 0.9982300884955752, 0.8578249433399397, 0.84477620075717041, 0.84477620075717041, 0.76930086277738707, 0.90286083483436441, 841.0, 289.0, 2.0, 1128.0, 1130.0, 1130.0]]]


True

In [12]:
'''
cross validation NB
'''
classifier="NB"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    print np.array(final_seq_value).shape
    print len(positive_seq[0])
    final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in final_seq_value]
    print np.array(final_seq_value).shape

    y_final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in y_final_seq_value]

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    NB=GaussianNB()
    NB.fit(X_train,Y_train)
    y_pred=NB.predict(X_test)
    y_pred_prob=NB.predict_proba(X_test)
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier,ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

(2034, 39)
42
(2034, 37)
0.880530973451
ACC_all: 0.880530973451
(2034, 39)
42
(2034, 37)
0.858407079646
ACC_all: 1.7389380531
(2034, 39)
42
(2034, 37)
0.862831858407
ACC_all: 2.6017699115
(2034, 39)
42
(2034, 37)
0.871681415929
ACC_all: 3.47345132743
(2034, 39)
42
(2034, 37)
0.898230088496
ACC_all: 4.37168141593
(2034, 39)
42
(2034, 37)
0.907079646018
ACC_all: 5.27876106195
(2034, 39)
42
(2034, 37)
0.907079646018
ACC_all: 6.18584070796
(2034, 39)
42
(2034, 37)
0.911504424779
ACC_all: 7.09734513274
(2034, 39)
42
(2034, 37)
0.893805309735
ACC_all: 7.99115044248
(2034, 39)
42
(2034, 37)
0.951327433628
ACC_all: 8.94247787611
[[['NB', 0.89424778761061963, 0.9730233975033313, 0.8115044247787611, 0.8115044247787611, 0.976991150442478, 0.8899705403223754, 0.88405486929839106, 0.88405486929839106, 0.80041860023150557, 0.91159135406061553, 917.0, 213.0, 26.0, 1104.0, 1130.0, 1130.0]]]


True

In [13]:
'''
cross validation LR
'''
classifier="LR"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    print np.array(final_seq_value).shape
    print len(positive_seq[0])
    final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in final_seq_value]
    print np.array(final_seq_value).shape

    y_final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in y_final_seq_value]

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    
    logisReg = LogisticRegression()
    C_range = 10.0 ** np.arange(-4,3,1)
    grid_parame = dict(C=C_range)
    clf = GridSearchCV(logisReg, grid_parame, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    C=clf.best_params_['C']
    y_pred=clf.predict(X_test)
    y_pred_prob=clf.predict_proba(X_test)
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"C:"+str(C),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

(2034, 39)
42
(2034, 37)
0.884955752212
ACC_all: 0.884955752212
(2034, 39)
42
(2034, 37)
0.818584070796
ACC_all: 1.70353982301
(2034, 39)
42
(2034, 37)
0.889380530973
ACC_all: 2.59292035398
(2034, 39)
42
(2034, 37)
0.836283185841
ACC_all: 3.42920353982
(2034, 39)
42
(2034, 37)
0.889380530973
ACC_all: 4.3185840708
(2034, 39)
42
(2034, 37)
0.898230088496
ACC_all: 5.21681415929
(2034, 39)
42
(2034, 37)
0.884955752212
ACC_all: 6.1017699115
(2034, 39)
42
(2034, 37)
0.87610619469
ACC_all: 6.97787610619
(2034, 39)
42
(2034, 37)
0.867256637168
ACC_all: 7.84513274336
(2034, 39)
42
(2034, 37)
0.924778761062
ACC_all: 8.76991150442
[[['LRC:10.0', 0.876991150442478, 0.9650902325060648, 0.7831858407079645, 0.7831858407079645, 0.9707964601769911, 0.871481025160769, 0.86373119972088741, 0.86373119972088741, 0.76860314024688126, 0.90017385856370891, 885.0, 245.0, 33.0, 1097.0, 1130.0, 1130.0]]]


True

In [14]:
'''
cross validation KNN
'''
classifier="KNN"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    print np.array(final_seq_value).shape
    print len(positive_seq[0])
    final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in final_seq_value]
    print np.array(final_seq_value).shape

    y_final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in y_final_seq_value]

    

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    knn = KNeighborsClassifier()
    #设置k的范围
    k_range = list(range(1,10))
    leaf_range = list(range(1,2))
    weight_options = ['uniform','distance']
    algorithm_options = ['auto','ball_tree','kd_tree','brute']
    param_gridknn = dict(n_neighbors = k_range,weights = weight_options,algorithm=algorithm_options,leaf_size=leaf_range)
    clf = GridSearchCV(knn, param_gridknn, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    n_neighbors=clf.best_params_['n_neighbors']
    weights=clf.best_params_['weights']
    algorithm=clf.best_params_['algorithm']
    leaf_size=clf.best_params_['leaf_size']
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"n_neighbors:"+str(n_neighbors)+"weights:"+str(weights)+"algorithm:"+str(algorithm)+"leaf_size:"+str(leaf_size),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

(2034, 39)
42
(2034, 37)
0.880530973451
ACC_all: 0.880530973451
(2034, 39)
42
(2034, 37)
0.840707964602
ACC_all: 1.72123893805
(2034, 39)
42
(2034, 37)
0.87610619469
ACC_all: 2.59734513274
(2034, 39)
42
(2034, 37)
0.871681415929
ACC_all: 3.46902654867
(2034, 39)
42
(2034, 37)
0.893805309735
ACC_all: 4.36283185841
(2034, 39)
42
(2034, 37)
0.91592920354
ACC_all: 5.27876106195
(2034, 39)
42
(2034, 37)
0.87610619469
ACC_all: 6.15486725664
(2034, 39)
42
(2034, 37)
0.902654867257
ACC_all: 7.05752212389
(2034, 39)
42
(2034, 37)
0.87610619469
ACC_all: 7.93362831858
(2034, 39)
42
(2034, 37)
0.924778761062
ACC_all: 8.85840707965
[[['KNNn_neighbors:8weights:distancealgorithm:autoleaf_size:1', 0.88584070796460179, 0.9768826681359382, 0.7911504424778762, 0.7911504424778762, 0.9805309734513272, 0.8801636459899893, 0.87305128653428599, 0.87305128653428599, 0.78708691245388385, 0.90349518364789727, 894.0, 236.0, 22.0, 1108.0, 1130.0, 1130.0]]]


True

In [15]:
'''
cross validation XGBoost
'''
classifier="XGBoost"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    
    print np.array(final_seq_value).shape
    print len(positive_seq[0])
    final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in final_seq_value]
    print np.array(final_seq_value).shape

    y_final_seq_value=[e[0:((len(positive_seq[0])-1)/2-1)]+e[((len(positive_seq[0])-1)/2+1):] for e in y_final_seq_value]
    

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    parameters = [{'n_estimators':range(1,10,1),
                      'max_depth':range(2,10),
                      'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
                      'subsample':[0.75,0.8,0.85,0.9]
                      }]
    clf = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs=1, scoring='accuracy')
    clf.fit(X_train, Y_train)
    n_estimators=clf.best_params_['n_estimators']
    max_depth=clf.best_params_['max_depth']
    learning_rate=clf.best_params_['learning_rate']
    subsample=clf.best_params_['subsample']
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"learning_rate:"+str(learning_rate)+"subsample:"+str(subsample),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

(2034, 39)
42
(2034, 37)
0.880530973451
ACC_all: 0.880530973451
(2034, 39)
42
(2034, 37)
0.862831858407
ACC_all: 1.74336283186
(2034, 39)
42
(2034, 37)
0.898230088496
ACC_all: 2.64159292035
(2034, 39)
42
(2034, 37)
0.893805309735
ACC_all: 3.53539823009
(2034, 39)
42
(2034, 37)
0.898230088496
ACC_all: 4.43362831858
(2034, 39)
42
(2034, 37)
0.920353982301
ACC_all: 5.35398230088
(2034, 39)
42
(2034, 37)
0.920353982301
ACC_all: 6.27433628319
(2034, 39)
42
(2034, 37)
0.933628318584
ACC_all: 7.20796460177
(2034, 39)
42
(2034, 37)
0.91592920354
ACC_all: 8.12389380531
(2034, 39)
42
(2034, 37)
0.946902654867
ACC_all: 9.07079646018
[[['XGBoostn_estimators:9max_depth:9learning_rate:0.1subsample:0.85', 0.90707964601769908, 0.9957876066954883, 0.8176991150442477, 0.8176991150442477, 0.9964601769911503, 0.9022549168096896, 0.89718605846846322, 0.89718605846846322, 0.82818918063798463, 0.9057588691361893, 924.0, 206.0, 4.0, 1126.0, 1130.0, 1130.0]]]


True